# Model Comparison and Per-Attack Analysis

This notebook provides detailed analysis of trained models:
- Load saved models and results
- Per-attack-type performance analysis
- Confusion matrix visualizations
- Training history plots (deep learning models)
- Recommendations for next steps

**Prerequisites:** Run `02_train_all_models.ipynb` first

In [ ]:
import sys
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

from data.preprocessing import CICIDS2017Preprocessor
from models.isolation_forest import IsolationForestDetector
from models.one_class_svm import OneClassSVMDetector
from models.autoencoder import AutoencoderDetector
from models.vae import VAEDetector

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports complete")

In [ ]:
# Directories
DATA_DIR = Path.cwd().parent / 'data' / 'raw'
MODELS_DIR = Path.cwd().parent / 'models'
RESULTS_DIR = Path.cwd().parent / 'results'

print("✅ Directories configured")

## Load Results

In [ ]:
# Load all results
with open(RESULTS_DIR / 'isolation_forest_results.pkl', 'rb') as f:
    if_results = pickle.load(f)

with open(RESULTS_DIR / 'ocsvm_results.pkl', 'rb') as f:
    ocsvm_results = pickle.load(f)

with open(RESULTS_DIR / 'autoencoder_results.pkl', 'rb') as f:
    ae_results = pickle.load(f)

with open(RESULTS_DIR / 'vae_results.pkl', 'rb') as f:
    vae_results = pickle.load(f)

all_results = pd.DataFrame([if_results, ocsvm_results, ae_results, vae_results])

print("✅ Results loaded")
print("\nSummary:")
print(all_results[['model', 'f1', 'precision', 'recall', 'fp_rate', 'training_time']])

## Load Test Data for Per-Attack Analysis

In [ ]:
print("Loading Wednesday test data...")
df_wednesday = pd.read_csv(DATA_DIR / 'Wednesday-workingHours.pcap_ISCX.csv')
df_test = df_wednesday.sample(n=100000, random_state=42)

# Preprocess
preprocessor = CICIDS2017Preprocessor()
# Load the fitted preprocessor from training
X_test, y_test = preprocessor.fit_transform(df_test)  # Will need to load saved preprocessor in production

# Get attack types
attack_types = df_test[' Label'].values

print(f"✅ Test data loaded: {len(df_test)} samples")
print(f"\nAttack distribution:")
print(df_test[' Label'].value_counts())

## Load Models and Get Predictions

In [ ]:
# Load models
if_detector = IsolationForestDetector.load(MODELS_DIR / 'isolation_forest_final.pkl')
ocsvm_detector = OneClassSVMDetector.load(MODELS_DIR / 'ocsvm_final.pkl')
ae_detector = AutoencoderDetector.load(MODELS_DIR / 'autoencoder_final.h5')
vae_detector = VAEDetector.load(MODELS_DIR / 'vae_final.h5')

# Get predictions
print("Getting predictions...")
y_pred_if = if_detector.predict(X_test)
y_pred_ocsvm = ocsvm_detector.predict(X_test)
y_pred_ae = ae_detector.predict(X_test)
y_pred_vae = vae_detector.predict(X_test)

print("✅ Models loaded and predictions generated")

## Per-Attack Type Performance

In [ ]:
def per_attack_analysis(y_true, y_pred, attack_types, model_name):
    """Analyze performance per attack type."""
    df = pd.DataFrame({
        'true': y_true,
        'pred': y_pred,
        'attack_type': attack_types
    })
    
    results = []
    for attack in df['attack_type'].unique():
        mask = df['attack_type'] == attack
        y_t = df.loc[mask, 'true'].values
        y_p = df.loc[mask, 'pred'].values
        
        if attack == 'BENIGN':
            # For benign, we want correct classification (0)
            detection_rate = (y_p == 0).mean()
        else:
            # For attacks, we want detection (1)
            detection_rate = (y_p == 1).mean()
        
        results.append({
            'attack_type': attack,
            'count': mask.sum(),
            'detection_rate': detection_rate,
            'model': model_name
        })
    
    return pd.DataFrame(results).sort_values('detection_rate', ascending=False)

# Analyze each model
if_per_attack = per_attack_analysis(y_test, y_pred_if, attack_types, 'Isolation Forest')
ocsvm_per_attack = per_attack_analysis(y_test, y_pred_ocsvm, attack_types, 'One-Class SVM')
ae_per_attack = per_attack_analysis(y_test, y_pred_ae, attack_types, 'Autoencoder')
vae_per_attack = per_attack_analysis(y_test, y_pred_vae, attack_types, 'VAE')

# Combine all
all_per_attack = pd.concat([if_per_attack, ocsvm_per_attack, ae_per_attack, vae_per_attack])

print("Per-Attack Type Detection Rates:\n")
print(all_per_attack.to_string(index=False))

# Save
all_per_attack.to_csv(RESULTS_DIR / 'per_attack_analysis.csv', index=False)
print(f"\n💾 Per-attack analysis saved to results/per_attack_analysis.csv")

## Visualize Per-Attack Performance

In [ ]:
# Create pivot table for heatmap
pivot = all_per_attack.pivot(index='attack_type', columns='model', values='detection_rate')

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=True, fmt='.2%', cmap='RdYlGn', vmin=0, vmax=1, cbar_kws={'label': 'Detection Rate'})
plt.title('Per-Attack Type Detection Rates Heatmap', fontsize=14, fontweight='bold')
plt.ylabel('Attack Type')
plt.xlabel('Model')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'per_attack_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Heatmap saved to results/per_attack_heatmap.png")

## Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

models = [
    ('Isolation Forest', y_pred_if),
    ('One-Class SVM', y_pred_ocsvm),
    ('Autoencoder', y_pred_ae),
    ('VAE', y_pred_vae)
]

for idx, (name, y_pred) in enumerate(models):
    ax = axes[idx // 2, idx % 2]
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['BENIGN', 'ATTACK'],
                yticklabels=['BENIGN', 'ATTACK'])
    ax.set_title(f'{name}\nF1={all_results[all_results["model"]==name]["f1"].values[0]:.4f}', 
                fontweight='bold')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'all_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Confusion matrices saved to results/all_confusion_matrices.png")

## Training History (Deep Learning Models)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Autoencoder
ae_history = ae_results['history']
axes[0].plot(ae_history['loss'], label='Training Loss')
axes[0].plot(ae_history['val_loss'], label='Validation Loss')
axes[0].set_title('Autoencoder Training History', fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# VAE
vae_history = vae_results['history']
axes[1].plot(vae_history['loss'], label='Training Loss')
axes[1].plot(vae_history['val_loss'], label='Validation Loss')
axes[1].set_title('VAE Training History', fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Training history saved to results/training_history.png")

## Key Insights and Recommendations

In [ ]:
best_model = all_results.loc[all_results['f1'].idxmax()]
worst_model = all_results.loc[all_results['f1'].idxmin()]

print("="*80)
print("KEY INSIGHTS")
print("="*80)

print(f"\n🏆 Best Model: {best_model['model']}")
print(f"   F1 Score: {best_model['f1']:.4f}")
print(f"   Precision: {best_model['precision']:.4f}")
print(f"   Recall: {best_model['recall']:.4f}")
print(f"   False Positive Rate: {best_model['fp_rate']:.2%}")
print(f"   Training Time: {best_model['training_time']:.2f}s")

print(f"\n📊 Worst Model: {worst_model['model']}")
print(f"   F1 Score: {worst_model['f1']:.4f}")

# Quality gate
print(f"\n🎯 Quality Gate (F1 >= 0.85):")
if best_model['f1'] >= 0.85:
    print(f"   ✅ PASSED - Ready for production!")
else:
    gap = 0.85 - best_model['f1']
    print(f"   ❌ NOT MET - Need improvement of {gap:.4f}")

# Hardest attacks to detect
attack_avg = all_per_attack[all_per_attack['attack_type'] != 'BENIGN'].groupby('attack_type')['detection_rate'].mean()
hardest_attacks = attack_avg.nsmallest(3)

print(f"\n⚠️  Hardest Attacks to Detect:")
for attack, rate in hardest_attacks.items():
    print(f"   - {attack}: {rate:.2%} average detection")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")
if best_model['f1'] >= 0.85:
    print(f"   1. Deploy {best_model['model']} to production")
    print(f"   2. Monitor false positive rate ({best_model['fp_rate']:.2%})")
    print(f"   3. Set up alerting pipeline")
    print(f"   4. Build REST API for real-time detection")
else:
    print(f"   1. Try ensemble: Combine OCSVM (precision) + VAE (recall)")
    print(f"   2. Hyperparameter tuning: Grid search on VAE latent_dim and kl_weight")
    print(f"   3. Train on full dataset: 530K samples vs current 50K")
    print(f"   4. Feature engineering: Add temporal/statistical aggregations")
    print(f"   5. Test on other attack types: Tuesday (Brute Force), Thursday (Web), Friday (PortScan)")

print("\n" + "="*80)

## Export Summary Report

In [ ]:
# Create summary report
report = f"""
NETWORK INTRUSION DETECTION - TRAINING SUMMARY REPORT
====================================================

Training Date: {pd.Timestamp.now()}
Training Data: Monday BENIGN (50,000 samples)
Test Data: Wednesday DoS/DDoS (100,000 samples)

MODEL PERFORMANCE
-----------------
{all_results[['model', 'f1', 'precision', 'recall', 'fp_rate', 'training_time']].to_string(index=False)}

BEST MODEL: {best_model['model']}
  F1 Score: {best_model['f1']:.4f}
  Precision: {best_model['precision']:.4f}
  Recall: {best_model['recall']:.4f}
  FP Rate: {best_model['fp_rate']:.2%}
  Training Time: {best_model['training_time']:.2f}s

QUALITY GATE (F1 >= 0.85): {'PASSED ✅' if best_model['f1'] >= 0.85 else f"NOT MET ❌ (gap: {0.85-best_model['f1']:.4f})"}

HARDEST ATTACKS TO DETECT
-------------------------
{chr(10).join([f"  - {attack}: {rate:.2%}" for attack, rate in hardest_attacks.items()])}

FILES GENERATED
---------------
  - models/isolation_forest_final.pkl
  - models/ocsvm_final.pkl
  - models/autoencoder_final.h5
  - models/vae_final.h5
  - results/final_comparison.csv
  - results/per_attack_analysis.csv
  - results/per_attack_heatmap.png
  - results/all_confusion_matrices.png
  - results/training_history.png

NEXT STEPS
----------
""" + (f"""
  1. Deploy {best_model['model']} to production
  2. Build REST API for real-time detection
  3. Set up monitoring and alerting
""" if best_model['f1'] >= 0.85 else f"""
  1. Try ensemble methods
  2. Hyperparameter tuning
  3. Train on full dataset (530K samples)
  4. Feature engineering
""")

# Save report
with open(RESULTS_DIR / 'TRAINING_SUMMARY.txt', 'w') as f:
    f.write(report)

print(report)
print(f"\n💾 Summary report saved to results/TRAINING_SUMMARY.txt")